# Đọc file và kiểm tra

In [150]:
import os 


sound_folder = 'Sound/'
folder_list = os.listdir(sound_folder)

for folder in folder_list:
    print()
    print("Folder: " + folder)
    for file in os.listdir(sound_folder + folder + '/'):

        print(file)


Folder: Bad
Bad Hipvoice-converted.wav
Bad Kienvoice-converted.wav
Bad Nghiavoice-converted.wav
Bad Tienvoice-converted.wav
Bad TrungDoanvoice-converted.wav
Bad TrungMaivoice-converted.wav

Folder: Bye
Bye Hip's voice-converted.wav
Bye Kienvoice-converted.wav
bye Nghia's voice-converted.wav
Bye Tien's voice-converted.wav
bye TrungDoan's voice-converted.wav
Bye TrungMai_s voice-converted.wav

Folder: Good
Good Hip's voice.wav
Good Kienvoice.wav
good Nghia's voice.wav
Good Tien's voice.wav
good TrungDoan's voice.wav
Good TrungMai_s voice.wav

Folder: Greate
Great Hip's voice-converted.wav
Great Kienvoice-converted.wav
greate Nghia's voice-converted.wav
Greate Tien's voice-converted.wav
greate TrungDoan's voice-converted.wav
Greate TrungMai_s voice-converted.wav

Folder: Hello
Hello Hip's voice-converted.wav
hello Kienvoice-converted.wav
hello Nghia's voice-converted.wav
Hello Tien's voice-converted.wav
hello TrungDoan's voice-converted.wav
Hello TrungMai_s voice-converted.wav

Folder: H

# Biến đổi đầu vào và gắn nhãn

In [155]:
import librosa


#create label_list and mfcc_list
mfcc_list = []
label_list = []


# each folder is label because each label have many file sound
for folder in folder_list:
    for file in os.listdir(sound_folder + folder + '/'):
        y, sr = librosa.load(os.path.join(sound_folder, folder, file), sr = None)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=12)

        label_list.append(folder)
        mfcc_list.append(mfcc)

print(len(label_list))
print(mfcc_list)


        

64
[array([[-6.90279419e+02, -6.58708374e+02, -5.87949341e+02,
        -5.35774353e+02, -4.98194275e+02, -4.81024445e+02,
        -4.81043304e+02, -4.83758148e+02, -4.84294586e+02,
        -4.95541138e+02, -5.08596771e+02, -5.16729065e+02,
        -5.20040405e+02, -5.14174561e+02, -5.06213898e+02,
        -5.01758881e+02, -4.97519867e+02, -4.69040283e+02,
        -4.25906769e+02, -4.09185059e+02, -4.06512512e+02,
        -3.99852081e+02, -4.01987579e+02, -4.07010193e+02,
        -4.17520874e+02, -4.30480164e+02, -4.45729401e+02,
        -4.49843170e+02, -2.66608398e+02, -1.87679901e+02,
        -1.88300751e+02, -2.07784256e+02, -2.08466400e+02,
        -2.01426575e+02, -1.91857742e+02, -1.85697052e+02,
        -1.95610840e+02, -2.13588715e+02, -2.16894852e+02,
        -2.24726578e+02, -2.51109299e+02, -3.43054047e+02,
        -4.52752777e+02, -4.80070435e+02, -4.90234253e+02,
        -4.87444885e+02, -4.90181061e+02, -4.93589478e+02,
        -4.94958588e+02, -3.58453766e+02, -3.1405835

# Sử lý dữ liệu đầu vào

In [152]:
import numpy as np

# Số đặc trưng mong muốn
desired_num_features = 200
processed_mfcc_list = []
# Cắt hoặc zero-padding các khung MFCC để có cùng số đặc trưng
def process_mfcc(mfcc):
    if mfcc.shape[1] > desired_num_features:
        mfcc = mfcc[:, :desired_num_features]
    elif mfcc.shape[1] < desired_num_features:
        num_missing_features = desired_num_features - mfcc.shape[1]
        mfcc = np.pad(mfcc, ((0, 0), (0, num_missing_features)), mode='constant')
    return mfcc


for mfcc in mfcc_list:
    processed_mfcc = process_mfcc(mfcc)
    processed_mfcc_list.append(processed_mfcc)


    
for mfcc in processed_mfcc_list:
    print(mfcc.shape)

(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)
(12, 200)


In [153]:

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(label_list)

one_hot_labels  = to_categorical(encoded_labels)

label_arr = np.asarray(encoded_labels)
mfcc_arr = np.asarray(processed_mfcc_list)


print(label_arr)
print(len(label_arr))


[0 0 0 0 0 0 1 1 1 1 1 1 2 2 2 2 2 2 3 3 3 3 3 3 4 4 4 4 4 4 5 5 5 5 5 5 6
 6 6 6 6 6 6 7 7 7 7 7 7 7 8 8 8 8 8 8 8 9 9 9 9 9 9 9]
64


# Huấn luyện bằng mô hình RNN


In [154]:
import tensorflow as tf


# label_numeric = [i for i, _ in enumerate(label_list)]
# label_arr = np.asarray(label_numeric)
# mfcc_arr = np.asarray(processed_mfcc_list)



model = tf.keras.Sequential([
    tf.keras.layers.LSTM(32),  # Sử dụng LSTM với 64 units
    
    tf.keras.layers.Dense(10, activation='relu')  
])

# Biên dịch mô hình
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Huấn luyện mô hình
model.fit(mfcc_arr, label_arr, epochs=10, batch_size=8, validation_split=0.1)

Epoch 1/10
8/8 [==============================] - 3s 97ms/step - loss: 9.0419 - accuracy: 0.0702 - val_loss: 8.4729 - val_accuracy: 0.0000e+00
Epoch 2/10
8/8 [==============================] - 0s 13ms/step - loss: 7.5579 - accuracy: 0.2632 - val_loss: 12.6350 - val_accuracy: 0.0000e+00
Epoch 3/10
8/8 [==============================] - 0s 11ms/step - loss: 6.4428 - accuracy: 0.3684 - val_loss: 14.6178 - val_accuracy: 0.0000e+00
Epoch 4/10
8/8 [==============================] - 0s 11ms/step - loss: 6.0364 - accuracy: 0.3860 - val_loss: 14.6598 - val_accuracy: 0.0000e+00
Epoch 5/10
8/8 [==============================] - 0s 11ms/step - loss: 5.6480 - accuracy: 0.4386 - val_loss: 14.8481 - val_accuracy: 0.0000e+00
Epoch 6/10
8/8 [==============================] - 0s 12ms/step - loss: 4.9674 - accuracy: 0.4912 - val_loss: 16.3073 - val_accuracy: 0.0000e+00
Epoch 7/10
8/8 [==============================] - 0s 11ms/step - loss: 4.6705 - accuracy: 0.5439 - val_loss: 16.1764 - val_accuracy: 0.00